<a href="https://colab.research.google.com/github/lhmin0614/Randomly_Wired_NN/blob/main/Graph_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and install libraries

In [ ]:
import math
import numpy as np

#  ER graph
parameter: n (node 개수), p (연결될 확률)

모든 node의 조합은 p의 확률로 연결됨.



In [ ]:
def er_graph(n, p):

  if p < math.log(n) / n:
      print("p is too small for given n.")
      # p> ln(n)/n이어야 single connected component graph가 될 확률이 높다.

  edges = list()
  # n by n random array
  # rand[i][j] == the probability of the edge(i,j) connected.
  rand = np.random.uniform(0.0, 1.0, size=(n, n))

  for i in range(n):
      for j in range(i+1, n):
          if rand[i][j] < p:
              edges.append((i, j)) 

  return edges

## BA graph
parameter : m (initial node 개수), n (최종 node 개수)

기존 node에 새로운 node를 연결할 때 연결확률은 기존 node의 degree에 비례한다.

In [ ]:
def ba_graph(n, m):

  assert 1 <= m < n, "m must be smaller than n."

  edges = list()
  deg = np.zeros(n)

  for i in range(m, n):
      if i == m:
          for j in range(i):
              edges.append((j, i))
              deg[j] += 1
              deg[i] += 1
          continue

      connection = np.random.choice(range(n), size=m, replace=False, p=deg/np.sum(deg))
      for cnt in connection:
          edges.append((cnt, i))
          deg[cnt] += 1
          deg[i] += 1

  edges.sort()
  
  return edges


## WS graph
parameter : n (# of node), k(# of initial connected neighbors per node), p(rewiring probability)

Every edge is rewired with probability p, starts from regular graph.

In [ ]:
def ws_graph(n, k, p):
  
  if k % 2 != 0:                            # k should be even
    k = k - 1

  init_edges = [[] for i in range(n)]
  rand = np.random.uniform(0.0, 1.0, (n, k//2))
  edges = list()

  for i in range(n):                        # each node is connected to k neighbor nodes.
    for j in range(k//2):
      jj = (i + j + 1) % n
      init_edges[i].append(jj)
      init_edges[jj].append(i)

  for i in range(n):
    for j in range(k//2):
      jj = (i + j + 1) % n
      jjj = jj
      if rand[i][j] < p:
        rand_list = list(set(range(n)) - set(init_edges[i]))
        jjj = np.random.choice(rand_list, 1)[0]
        init_edges[i].append(jjj)
        init_edges[jjj].append(i)
        init_edges[i].remove(jj)
        init_edges[jj].remove(i)
      edges.append(tuple(sorted([i, jjj])))
  
  return sorted(edges)

In [ ]:
print(er_graph(20, 0.2))
print(ba_graph(20, 4))
print(ws_graph(20, 4, 0.4))

[(0, 5), (0, 6), (0, 11), (0, 13), (0, 14), (0, 19), (1, 5), (1, 6), (1, 8), (1, 9), (1, 17), (2, 9), (2, 12), (2, 14), (2, 16), (2, 17), (3, 5), (3, 6), (3, 7), (3, 9), (3, 14), (3, 18), (4, 6), (4, 9), (4, 12), (4, 13), (4, 17), (4, 19), (5, 6), (5, 11), (5, 19), (6, 7), (6, 11), (6, 13), (6, 14), (6, 17), (7, 11), (7, 17), (8, 11), (8, 15), (8, 16), (9, 11), (9, 16), (11, 16), (13, 18)]
[(0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 13), (0, 14), (0, 15), (0, 16), (1, 4), (1, 7), (1, 8), (1, 10), (1, 11), (1, 14), (1, 16), (1, 17), (1, 18), (2, 4), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7), (3, 10), (3, 12), (4, 5), (4, 6), (4, 8), (4, 9), (4, 10), (4, 11), (4, 13), (4, 17), (5, 6), (5, 7), (5, 8), (5, 9), (5, 11), (5, 14), (5, 17), (6, 19), (7, 9), (7, 11), (7, 12), (7, 15), (7, 17), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 18), (8, 19), (9, 19), (10, 12), (10, 15), (11, 13), (13, 19), (15, 16), (15, 18), (16, 18)]
[(0, 1), (0, 2), (0, 12), (0, 13), (0, 18), (1,

## Input and Output Nodes

Input nodes: Nodes that only send data

Output nodes: Nodes that only recieve data

In [ ]:
def in_out_nodes(n, edges):
  vertices = set(range(n))
  starts, ends = set(), set()
  for edge in edges:
    starts.add(edge[0])
    ends.add(edge[1])
  
  inputs = sorted(list(vertices - ends))
  outputs = sorted(list(vertices - starts))
  return inputs, outputs

er1 = er_graph(20, 0.2)
ba1 = ba_graph(20, 4)
ws1 = ws_graph(20, 4, 0.4)

print(er1)
print(ba1)
print(ws1)

print(in_out_nodes(20, er1))
print(in_out_nodes(20, ba1))
print(in_out_nodes(20, ws1))

[(0, 11), (1, 2), (1, 6), (1, 12), (1, 14), (1, 19), (2, 5), (2, 9), (2, 14), (3, 5), (3, 10), (3, 13), (3, 16), (3, 17), (4, 5), (4, 6), (4, 10), (5, 7), (5, 8), (5, 10), (5, 11), (5, 14), (5, 19), (6, 9), (6, 17), (7, 8), (7, 10), (7, 17), (7, 19), (8, 9), (8, 13), (9, 13), (9, 14), (10, 11), (10, 13), (10, 14), (10, 15), (11, 12), (11, 16), (12, 13), (13, 15), (15, 18), (16, 19), (17, 18), (18, 19)]
[(0, 4), (1, 4), (1, 5), (1, 6), (1, 7), (1, 9), (2, 4), (2, 5), (2, 6), (2, 10), (3, 4), (3, 5), (3, 8), (3, 9), (3, 10), (3, 12), (3, 13), (3, 15), (3, 19), (4, 5), (4, 6), (4, 7), (4, 8), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 19), (5, 6), (5, 7), (5, 8), (5, 10), (5, 11), (5, 13), (5, 17), (5, 18), (6, 7), (6, 8), (6, 9), (6, 14), (6, 15), (6, 16), (6, 18), (7, 9), (7, 11), (7, 14), (7, 16), (7, 17), (8, 19), (9, 10), (9, 15), (10, 11), (10, 12), (11, 12), (11, 13), (11, 14), (11, 18), (12, 17), (15, 16), (15, 18), (16, 17), (18, 19)]
[(0, 1), (0, 2), (0, 9), (0, 1